In [1]:
%load_ext autoreload
%autoreload 2
import sys
print(sys.executable)
import socket
print(socket.gethostname())
import os
os.chdir("/home/icb/alioguz.can/projects/campa_pt")

/home/icb/alioguz.can/miniconda3/envs/campa_pt/bin/python
cpusrv30.scidom.de


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
TF_ENABLE_ONEDNN_OPTS=0

In [3]:
from pathlib import Path
from campa.tl import (
    Cluster,
    Estimator,
    TorchEstimator,
    Predictor,
    TorchPredictor,
    Experiment,
    TorchExperiment,
    ModelComparator,
    run_experiments,
    run_torch_experiments
)
from campa.data import MPPData
from campa.utils import init_logging
from campa.constants import campa_config
# init logging with level INFO=20, WARNING=30
init_logging(level=30)
# read correct campa_config -- created with setup.ipynb
CAMPA_DIR = Path.cwd()
campa_config.config_fname = CAMPA_DIR / "notebooks/params/campa.ini"
print(campa_config)

/home/icb/alioguz.can/miniconda3/envs/campa_pt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-11 16:44:43.052934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-11 16:44:54.938237: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-11 16:45:46.768108: W tensorflow/compiler/xla/stream_exe

Reading config from /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini
CAMPAConfig (fname: /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini)
EXPERIMENT_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments
BASE_DATA_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_data
CO_OCC_CHUNK_SIZE: 10000000.0
data_config/exampledata: /home/icb/alioguz.can/projects/campa_pt/notebooks/params/ExampleData_constants.py



In [4]:
# import tensorflow as tf
# from tensorflow.keras import backend as K
# import torch
# torch.cuda.empty_cache()
# # Clear the current session
# K.clear_session()

# # Optionally, reset the default graph
# tf.compat.v1.reset_default_graph()

In [5]:
# get Experiments from config
exps = Experiment.get_experiments_from_config("notebooks/params/example_experiment_params.py")

In [6]:
# run_experiments(exps, mode="trainval")

In [7]:
exp = exps[0]
print("Experiment name:", exp.name)
print("Experiment is stored in:", exp.full_path)

Experiment name: VAE
Experiment is stored in: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE


In [8]:
est = Estimator(exps[0])

2025-01-11 16:48:33.783180: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-01-11 16:48:33.799446: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-01-11 16:48:33.800386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cpusrv30.scidom.de): /proc/driver/nvidia/version does not exist
2025-01-11 16:48:33.974215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


latent before reparam (None, 32)
latent after reparam (None, 16)
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 3, 34)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 3, 3, 32)     1120        ['input_1[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 288)          0           ['conv2d[0][0]']                 
                                                                                                  
 dense (Dense)                  (None, 32)           9248        ['flatten[0][0]']                
                           

In [10]:
est.train_model()

4/4 [==============================] - 0s 45ms/step - loss: 5494.1187 - decoder_loss: 5494.1025 - latent_loss: 0.0161 - decoder_mean_squared_error: 0.8435 - latent_kl_loss: 0.0162 - val_loss: 5567.1948 - val_decoder_loss: 5567.1758 - val_latent_loss: 0.0191 - val_decoder_mean_squared_error: 0.8391 - val_latent_kl_loss: 0.0191


,loss,decoder_loss,latent_loss,decoder_mean_squared_error,latent_kl_loss,val_loss,val_decoder_loss,val_latent_loss,val_decoder_mean_squared_error,val_latent_kl_loss
epoch,,,,,,,,,,
0,5494.118652,5494.102539,0.01611,0.843513,0.016176,5567.194824,5567.175781,0.019061,0.839096,0.019061


In [21]:
print(est.model.model.summary())

Model: "VAEModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 3, 3, 34)]   0           []                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 3, 3, 32)     1120        ['input_7[0][0]']                
                                                                                                  
 flatten_3 (Flatten)            (None, 288)          0           ['conv2d_3[0][0]']               
                                                                                                  
 dense_8 (Dense)                (None, 32)           9248        ['flatten_3[0][0]']              
                                                                                           

In [11]:
torch_exps = TorchExperiment.get_experiments_from_config("notebooks/params/example_experiment_params_torch.py")

In [12]:
est_torch = TorchEstimator(torch_exps[0])
print(est_torch.model)
est_torch.model.total_trainable_params()

VAEModelTorch(
  (encoder_input): ModuleList()
  (decoder_input): ModuleList()
  (encoder): Sequential(
    (0): Conv2d(34, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=288, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=16, bias=True)
    (6): ReLU()
    (7): Linear(in_features=16, out_features=32, bias=True)
  )
  (latent): Linear(in_features=16, out_features=32, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=34, bias=True)
  )
)
12018


In [13]:
est_torch.train_model()

/home/icb/alioguz.can/miniconda3/envs/campa_pt/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py:172: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


[{'epoch': 0, 'loss': 18862.36279296875, 'val_loss': 4765.98974609375}]

In [16]:
pred_torch = TorchPredictor(torch_exps[0])
pred_torch.evaluate_model()

In [17]:
results_folder = os.path.join(pred_torch.exp.full_path, f"results_epoch{pred_torch.est.epoch:03d}")
print("Results folder", results_folder)
print(os.listdir(results_folder))

Results folder /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch000
['val', 'val_imgs']


In [18]:
print(MPPData.from_data_dir(os.path.join(results_folder, "val"), data_config="ExampleData"))

MPPData for ExampleData (123 mpps with shape (3, 3, 34) from 8 objects). Data keys: ['x', 'obj_ids', 'y', 'labels', 'mpp', 'conditions', 'latent'].


In [19]:
cl = Cluster.from_exp_split(torch_exps[0])

Cannot read with memmap:  /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch001/val/clustering.npy


AssertionError: 

In [ ]:
print(cl.config["leiden_resolution"])
cl.create_clustering()

In [20]:
# run_torch_experiments(torch_exps, mode="trainval")